In [43]:
import requests
from bs4 import BeautifulSoup
import netCDF4
import pandas as pd
import os



## This cell navigates to the CUDEM data catalog for the 1/9 arcsecond data and extracts the names of all downloadable files. It then iterates on each name, adding it to a base url and downloading it to the input directory  with wget

In [ ]:
#Download CUDEM Files


os.chdir('/home/will/Desktop/USGS/CUDEM/Nc')
#Extract link to file names from 1/9 arc second resolution server


def get_url_paths(url, ext='', params={}):
    response = requests.get(url, params=params)
    if response.ok:
        response_text = response.text
    else:
        return response.raise_for_status()
    soup = BeautifulSoup(response_text, 'html.parser')
    parent = [url + node.get('href') for node in soup.find_all('a') if node.get('href').endswith(ext)]
    return parent

url = 'https://www.ngdc.noaa.gov/thredds/catalog/tiles/tiled_19as/catalog.html'
ext = 'nc'
result = get_url_paths(url, ext)

#Make links into df
link_df = pd.DataFrame(columns = ['link'])
link_df['link']=result

#split link field by '/' delimiter to get file name

link_df['filename']=link_df['link'].str.split('/', expand=True)[9]

#download iterating through each file name

base_url='https://www.ngdc.noaa.gov/thredds/fileServer/tiles/tiled_19as/' #base url for download
i=0
for i in range(0,len(link_df)):
    file_name=link_df['filename'][i]
    dwnld_link=base_url+file_name
    !wget {dwnld_link}
    i=i+1
    



--2022-01-10 14:46:01--  https://www.ngdc.noaa.gov/thredds/fileServer/tiles/tiled_19as/ncei19_n49x00_w123x00_2019v2.nc
Resolving www.ngdc.noaa.gov (www.ngdc.noaa.gov)... 140.172.190.1
Connecting to www.ngdc.noaa.gov (www.ngdc.noaa.gov)|140.172.190.1|:443... connected.
HTTP request sent, awaiting response... 200 200
Length: 147193634 (140M) [application/x-netcdf]
Saving to: ‘ncei19_n49x00_w123x00_2019v2.nc’

ncei19_n49x00_w123x 100%[===================>] 140.37M  2.63MB/s    in 57s     

2022-01-10 14:46:58 (2.46 MB/s) - ‘ncei19_n49x00_w123x00_2019v2.nc’ saved [147193634/147193634]

--2022-01-10 14:46:59--  https://www.ngdc.noaa.gov/thredds/fileServer/tiles/tiled_19as/ncei19_n49x00_w122x75_2019v2.nc
Resolving www.ngdc.noaa.gov (www.ngdc.noaa.gov)... 140.172.190.1
Connecting to www.ngdc.noaa.gov (www.ngdc.noaa.gov)|140.172.190.1|:443... connected.
HTTP request sent, awaiting response... 200 200
Length: 157836487 (151M) [application/x-netcdf]
Saving to: ‘ncei19_n49x00_w122x75_2019v2.nc’



--2022-01-10 15:03:08--  https://www.ngdc.noaa.gov/thredds/fileServer/tiles/tiled_19as/ncei19_n47x00_w0124x25_2018v1.nc
Resolving www.ngdc.noaa.gov (www.ngdc.noaa.gov)... 140.172.190.1
Connecting to www.ngdc.noaa.gov (www.ngdc.noaa.gov)|140.172.190.1|:443... connected.
HTTP request sent, awaiting response... 200 200
Length: 156289196 (149M) [application/x-netcdf]
Saving to: ‘ncei19_n47x00_w0124x25_2018v1.nc’

     ncei19_n47x00_  35%[======>             ]  52.87M  2.32MB/s    eta 45s    

## This cell converts all of the downloaded cudem nc's into .csv files with fields for latitude, longitude, depth, and crs

In [70]:
#Convert downloaded nc files to csv
csv_folder='/home/will/Desktop/USGS/CUDEM/'#enter path for csv folder

for filename in os.listdir(os.getcwd()):
    if filename.endswith(".nc"):
        nc = netCDF4.Dataset(os.path.join(os.getcwd(), filename), mode='r')
        file_name_no_ext=os. path. splitext(filename)[0]
        out_name=csv_folder+file_name_no_ext+'.csv'
        df = pd.DataFrame(columns = ['latitude','longitude','depth'])
        df['latitude']=nc.variables['lat'][:]
        df['longitude']= nc.variables['lon'][:]
        df['depth']=nc.variables['Band1'][:]
        df['crs']=nc.variables['crs'][:]
        df.to_csv(out_name)
        a=a+1        
    else:
        continue

## This cell combines all of the  csv's into one dataframe

In [88]:
#merge csv's into one df


os.chdir(csv_folder)

#create list of files in folder
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]

#combine all files in the list
combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames ])

cudem_df=combined_csv 

#round lat lon to 4th decimal place
cudem_df['latitude']=cudem_df['latitude'].round(decimals=3)
cudem_df['longitude']=cudem_df['longitude'].round(decimals=3)

#create lat lon combined column
cudem_df['latlon']= cudem_df['latitude'].map(str)+','+cudem_df['longitude'].map(str)

/home/will/anaconda3/envs/USGS/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3357: DtypeWarning: Columns (1,2,6,7,10,11,12,13,15,21,23,26,27,28) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


## This cell calls your sample data and converts it to a dataframe

In [68]:
#call in sample dataset

# from https://stackoverflow.com/questions/38511444/python-download-files-from-google-drive-using-url

def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)    

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

def save_response_content(response, destination):
    """
    response = filename for input
    destination = filename for output
    """    
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)


DATASET_ID = '1G9fuC_TjtwTr3JWA85gW7228_Ffxsw0G'


destination = csv_folder+'sample_data.csv'
download_file_from_google_drive(DATASET_ID, destination)
sample_df= pd.read_csv(destination)

/home/will/anaconda3/envs/USGS/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (1,2,6,7,10,11,12,13,15,21,23,26,27,28) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


## This cell merges both output dataframe, effectively adding a depth field to each sample that matches a CUDEM lat/lon point

In [89]:
#round sample lat lon to 4th decimal place
sample_df['latitude']=sample_df['latitude'].round(decimals=3)
sample_df['longitude']=sample_df['longitude'].round(decimals=3)


#create lat lon combined column
sample_df['latlon']= sample_df['latitude'].map(str)+','+sample_df['longitude'].map(str)

#merge dataframes on latlon column

merged_df=pd.merge(sample_df,cudem_df)